# Build a Crew to Tailor Job Applications



### The libraries are already installed,if you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached embedchain-0.1.128-py3-none-any.whl.metadata (9.2 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.31.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached lancedb-0.5.7-py3-none-any.whl.metadata (17 kB)
  Using cached pyright-1.1.398-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached langchain_core-0.1.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.7 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.
gradio 5.15.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.
langchain-chroma 0.2.1 requires langchain-core!=0.3.0,!=0.3.1,!=0.3.10,!=0.3.11,!=0.3.12,!=0.3.13,!=0.3.14,!=0.3.2,!=0.3.3,!=0.3.4,!=0.3.5,!=0.3.6,!=0.3.7,!=0.3.8,!=0.3.9,<0.4.0,>=0.2.43, but you have langchain-core 0.1.53 which is incompatible.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Access the keys
openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME")

# Print confirmation (DO NOT print actual API keys for security reasons)
print("API keys loaded successfully!")




API keys loaded successfully!


## crewAI Tools

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


In [6]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [7]:
# Define the Researcher Agent
researcher = Agent(
    role="Job Researcher",
    goal="Extract job details from job postings. Make sure to scrape the job posting details like Job description , Qualifications and other important details.",
    tools=[scrape_tool],  # Assign scraping tool
    verbose=True,
    backstory="An expert job market researcher who finds the best AI/ML roles."
)


In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
# research_task = Task(
#     description=(
#         "Analyze the job posting URL provided ({job_posting_url}) "
#         "to extract key skills, experiences, and qualifications "
#         "required. Use the tools to gather content and identify "
#         "and categorize the requirements."
#     ),
#     expected_output=(
#         "A structured list of job requirements, including necessary "
#         "skills, qualifications, and experiences."
#     ),
#     agent=researcher,
#     async_execution=True
# )



# Define the Task for the Agent
research_task = Task(
    description="Scrape job details from the provided URL: {job_posting_url}. Extract job title, company name, and job description , skills, Qualifications",
    expected_output="A structured list of job details (title, company, location, job description, requirements , qualifications , skills).",
    agent=researcher
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information.Dont add any skills that are not prsent inn resume. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job.The file should be openable and no errors should be prsent while opening the file."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
job_application_inputs = {
    'job_posting_url': 'https://www.naukri.com/job-listings-senior-data-scientist-artificial-intelligence-machine-learning-zettamine-labs-pvt-ltd-mumbai-4-to-6-years-170325909338?src=simjobsjd_bottom',
    'personal_writeup': """John is a highly skilled AI/ML Engineer with a strong background in machine learning, deep learning, and data science.
With expertise in Python, TensorFlow, PyTorch, and cloud-based AI solutions, he has successfully designed and deployed intelligent models that drive business impact.
John has experience working with large-scale datasets, building predictive analytics models, and developing AI-powered applications in industries ranging from finance to healthcare.
His ability to combine technical proficiency with strategic problem-solving allows him to create innovative AI solutions that enhance automation, efficiency, and decision-making.
Passionate about staying at the forefront of AI advancements, John continuously explores emerging technologies such as generative AI, reinforcement learning, and multimodal models.
He is an ideal candidate for roles that demand cutting-edge AI expertise, leadership in ML projects, and a vision for driving innovation through artificial intelligence.
"""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Job Researcher
 [INFO]: == Starting Task: Scrape job details from the provided URL: https://www.naukri.com/job-listings-senior-data-scientist-artificial-intelligence-machine-learning-zettamine-labs-pvt-ltd-mumbai-4-to-6-years-170325909338?src=simjobsjd_bottom. Extract job title, company name, and job description , skills, Qualifications


> Entering new CrewAgentExecutor chain...
I need to extract job details such as job title, company name, job description, skills, qualifications from the provided URL.

Action: Read website content
  Input: {"website_url": "https://www.naukri.com/job-listings-senior-data-scientist-artificial-intelligence-machine-learning-zettamine-labs-pvt-ltd-mumbai-4-to-6-years-170325909338?src=simjobsjd_bottom"}



Senior Data Scientist - Artificial Intelligence/Machine Learning at Zettamine Labs Pvt Ltd, Mumbai

Job Description:
Zettamine Labs Pvt Ltd is looking for a Senior Data Scientist with expertise in Artificial Intelligence and M

- Dislplay the generated `tailored_resume.md` file.

In [18]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

# Johnathan Doe

**Email:** johnathan.doe@example.com  
**Phone:** +1-234-567-8901  
**Website:** [www.johndoeportfolio.com](http://www.johndoeportfolio.com)  
**LinkedIn:** [linkedin.com/in/johndoe](http://linkedin.com/in/johndoe)  
**GitHub:** [github.com/johndoe](http://github.com/johndoe)  
**Location:** New York, USA  

---

## **Professional Summary**
A highly versatile and results-driven professional with over 15 years of experience across multiple domains, including Software Engineering, Data Science, Cybersecurity, Business Strategy, and Digital Marketing. Adept at designing and implementing complex systems, leading high-performing teams, and developing data-driven solutions for business growth. Passionate about technological innovation and continuous learning. Proven track record in optimizing operational efficiency, enhancing security, and driving revenue growth through strategic initiatives.

---

## **Technical Skills**

### **Programming Languages & Software Development**
- Proficient: Python, Java, C++, JavaScript, TypeScript, Swift, Kotlin
- Web Development: HTML, CSS, React, Angular, Vue.js, Node.js, Express.js, Django, Flask
- Mobile Development: Android (Kotlin/Java), iOS (Swift), React Native
- Backend Development: RESTful APIs, GraphQL, Microservices Architecture, Kubernetes, Docker, AWS Lambda
- Software Engineering Principles: SOLID Principles, Design Patterns, Agile, DevOps, CI/CD Pipelines

### **Data Science & Machine Learning**
- Data Analytics: Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn
- Machine Learning Algorithms: Regression, Classification, Clustering, Neural Networks, Random Forests
- Deep Learning: TensorFlow, Keras, PyTorch
- Data Engineering: Apache Spark, Hadoop, Airflow
- Database Management: SQL, PostgreSQL, MySQL, MongoDB, Cassandra

### **Cybersecurity & Ethical Hacking**
- Penetration Testing: Metasploit, Kali Linux, Burp Suite
- Cryptography: AES, RSA, SHA-256
- Network Security: Firewalls, IDS/IPS, VPNs
- Security Compliance: GDPR, HIPAA, ISO 27001
- Ethical Hacking: CEH Certified, Red Teaming, Blue Teaming

### **Business & Digital Marketing**
- Business Strategy: SWOT Analysis, Competitive Intelligence, Market Research
- SEO & SEM: Google Analytics, Ahrefs, SEMrush, Keyword Research
- Content Marketing: Blogging, Video Marketing, Social Media Marketing (Facebook, Instagram, LinkedIn, Twitter)
- Growth Hacking: A/B Testing, Conversion Rate Optimization (CRO)
- E-commerce: Shopify, WooCommerce, Magento

---

## **Professional Experience**

### **Chief Technology Officer (CTO) - Tech Innovators Inc.**  
*January 2018 - Present | New York, USA*
- Spearheaded the development and deployment of AI-driven cybersecurity solutions, reducing security breaches by 50%.
- Led a cross-functional team of 100+ developers, data scientists, and security analysts to implement scalable enterprise software solutions.
- Developed a machine learning-based fraud detection system that saved the company over $10 million in fraudulent transactions.
- Managed cloud infrastructure across AWS, Azure, and Google Cloud to optimize system performance and cost efficiency.
- Championed Agile methodologies, improving product release cycles by 40%.

### **Lead Data Scientist - FinTech Solutions**  
*July 2015 - December 2017 | San Francisco, USA*
- Designed predictive analytics models to enhance customer segmentation, increasing revenue by 25%.
- Developed natural language processing (NLP) algorithms to automate customer service chatbots, reducing human intervention by 60%.
- Built recommendation systems that personalized user experiences, leading to a 35% increase in engagement.
- Collaborated with product managers and business leaders to translate business objectives into actionable AI/ML strategies.

### **Senior Software Engineer - Global Tech Corp**  
*March 2012 - June 2015 | Chicago, USA*
- Engineered scalable RESTful APIs and microservices architecture for high-traffic applications with over 10 million daily active users.
- Optimized SQL queries and NoSQL databases, improving database performance by 70%.
- Led migration of monolithic applications to cloud-based microservices, reducing operational costs by 30%.
- Mentored junior developers, enhancing team productivity and knowledge-sharing.

### **Cybersecurity Consultant - SecureNet Solutions**  
*June 2010 - February 2012 | Washington, USA*
- Conducted penetration testing for Fortune 500 clients, identifying and mitigating critical vulnerabilities.
- Developed security policies and protocols, ensuring compliance with industry standards like ISO 27001 and GDPR.
- Implemented multi-layered authentication systems, strengthening security posture across enterprises.

---

## **Education**

**Master of Science in Computer Science**  
*Stanford University, 2010*

**Bachelor of Science in Information Technology**  
*Massachusetts Institute of Technology, 2008*

---

## **Certifications**
- AWS Certified Solutions Architect
- Certified Ethical Hacker (CEH)
- Google Data Engineer Certification
- PMI Agile Certified Practitioner (PMI-ACP)

---

## **Projects**

### **AI-Powered Resume Screener**
- Developed a machine learning model that screens job applicants based on job descriptions and ranks resumes accordingly.
- Achieved 90% accuracy in predicting the best candidates for various roles.

### **Blockchain-Based Identity Verification System**
- Created a decentralized identity verification system using Ethereum smart contracts.
- Enhanced security and reduced fraud cases in financial transactions.

### **E-commerce Growth Optimization**
- Led an A/B testing campaign that optimized checkout flows, increasing conversion rates by 15%.
- Utilized AI-driven analytics to personalize product recommendations, resulting in a 30% boost in sales.

---

## **Publications & Research**
- *"Advancements in AI-driven Cybersecurity Solutions"* - IEEE Security & Privacy Journal, 2021
- *"Optimizing Machine Learning Models for High-Frequency Trading"* - Data Science Review, 2020
- *"The Future of Blockchain in Secure Digital Identity"* - MIT Technology Review, 2019

---

## **Speaking Engagements**
- Keynote Speaker, AI Summit 2022 - *"The Role of AI in Cybersecurity"*
- Panelist, TechCrunch Disrupt 2021 - *"Scaling Startups with Cloud Computing"*
- Guest Lecturer, Harvard Business School - *"Data Science for Business Strategy"*

---

## **Volunteer Experience**
- **Tech for Good Initiative**: Mentored underprivileged youth in coding and software development.
- **Open Source Contributor**: Contributed to TensorFlow, Apache Spark, and Kubernetes projects.

---

## **Languages**
- English: Native
- Spanish: Fluent
- French: Intermediate

---

## **Hobbies & Interests**
- Writing technical blogs on AI and cybersecurity
- Open-source software development
- Marathon running and fitness
- Playing the piano and composing music


In [21]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

The updated resume content has effectively highlighted the candidate's qualifications and experiences relevant to the Senior Data Scientist position at Zettamine Labs Pvt Ltd.

- Dislplay the generated `interview_materials.md` file.

In [22]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Johnathan Doe:**

1. Can you elaborate on your experience in developing and deploying AI-driven cybersecurity solutions as the Chief Technology Officer at Tech Innovators Inc.? How did you reduce security breaches by 50%?
2. As a Lead Data Scientist at FinTech Solutions, you designed predictive analytics models to enhance customer segmentation. Can you walk us through the impact this had on revenue growth?
3. Your role as a Senior Software Engineer at Global Tech Corp involved migrating monolithic applications to cloud-based microservices. How did this transition improve operational efficiency and reduce costs?
4. Could you discuss your involvement in conducting penetration testing for Fortune 500 clients as a Cybersecurity Consultant at SecureNet Solutions? How did you identify and mitigate critical vulnerabilities?
5. In your AI-Powered Resume Screener project, how did you achieve 90% accuracy in predicting the best candidates for various roles based on job descriptions?
6. What motivated you to create a Blockchain-Based Identity Verification System using Ethereum smart contracts? How did this system enhance security and reduce fraud cases in financial transactions?
7. Can you share insights from your experience as a Keynote Speaker at the AI Summit 2022 on "The Role of AI in Cybersecurity"? What key points did you emphasize in your speech?
8. How do you see the future of blockchain in secure digital identity based on your publication in the MIT Technology Review?
9. Your volunteer experience in mentoring underprivileged youth in coding is commendable. How has this experience influenced your perspective on technology and education?
10. In your spare time, you enjoy writing technical blogs on AI and cybersecurity. How do you stay updated on the latest trends and advancements in these fields?

These questions and talking points are designed to help Johnathan Doe showcase his expertise, experiences, and achievements in alignment with the job requirements for the Senior Data Scientist position at Zettamine Labs Pvt Ltd.